# 集約
データの価値をなるべく損失せずに圧縮し、データの単位を変換できる処理

## データ数、種類数の算出

In [1]:
import sys

import numpy as np
import pandas as pd
sys.path.append("../src")
from data_loader import load_hotel_reserve

In [2]:
customer_tb, hotel_tb, reserve_tb = load_hotel_reserve()

In [3]:
customer_tb.head()

,customer_id,age,sex,home_latitude,home_longitude
0,c_1,41,man,35.092193,136.512347
1,c_2,38,man,35.325076,139.410551
2,c_3,49,woman,35.120543,136.511179
3,c_4,43,man,43.034868,141.240314
4,c_5,31,man,35.102661,136.523797


In [4]:
hotel_tb.head()

,hotel_id,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
0,h_1,26100,D,D-2,43.064569,141.511397,True
1,h_2,26400,A,A-1,35.715320,139.939446,True
2,h_3,41300,E,E-4,35.281572,136.988565,False
3,h_4,5200,C,C-3,38.431293,140.795615,False
4,h_5,13500,G,G-3,33.597291,130.533872,True


In [5]:
reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


## データ数、種類数の算出
ホテル単位で予約レコードをまとめ、`size`関数で予約数のカウント、`nunique`関数で顧客数のユニークカウント

In [6]:
result = reserve_tb \
  .groupby('hotel_id') \
  .agg({'reserve_id': 'count', 'customer_id': 'nunique'})

# reset_index関数によって、列番号を振り直す（inplace=Trueなので、直接resultを更新）
result.reset_index(inplace=True)
result.columns = ['hotel_id', 'rsv_cnt', 'cus_cnt']

In [7]:
result

,hotel_id,rsv_cnt,cus_cnt
0,h_1,10,10
1,h_10,3,3
2,h_100,20,19
3,h_101,17,17
4,h_102,13,13
...,...,...,...
295,h_95,13,13
296,h_96,13,13
297,h_97,16,16
298,h_98,17,16


# 合計値の算出
`sum`関数で合計の集約  
列名の変更は書き換える列が少ない場合は、DataFrameのcolumnsを書き換えるよりも`rename`を使う

In [8]:
# 集約単位をhotel_idとpeople_numの組み合わせを指定
# 集約したデータからtotal_priceを取り出し、sum関数に適用することで売上合計金額を算出
result = reserve_tb \
  .groupby(['hotel_id', 'people_num'])['total_price'] \
  .sum().reset_index()

# 売上合計金額の列名がtotal_priceになっているので、price_sumに変更
result.rename(columns={'total_price': 'price_sum'}, inplace=True)

In [9]:
result

,hotel_id,people_num,price_sum
0,h_1,1,156600
1,h_1,2,156600
2,h_1,3,391500
3,h_1,4,417600
4,h_10,1,11200
...,...,...,...
1154,h_98,3,793800
1155,h_98,4,453600
1156,h_99,1,179200
1157,h_99,2,448000


## 極値、代表値の算出

In [10]:
# Pythonのラムダ式をagg関数の集約処理に指定
# ラムダ式にはnumpy.percentileを指定しパーセントタイル値を算出（パーセントは20指定）
result = reserve_tb \
  .groupby('hotel_id') \
  .agg({'total_price': ['max', 'min', 'mean', 'median',
                        lambda x: np.percentile(x, q=20)]}) \
  .reset_index()
result.columns = ['hotel_id', 'price_max', 'price_min', 'price_mean',
                  'price_median', 'price_20per']

In [11]:
result

,hotel_id,price_max,price_min,price_mean,price_median,price_20per
0,h_1,208800,26100,112230.000000,104400.0,73080.0
1,h_10,67200,11200,42933.333333,50400.0,26880.0
2,h_100,57600,4800,27600.000000,28800.0,9600.0
3,h_101,168000,14000,75764.705882,56000.0,30800.0
4,h_102,72000,12000,32769.230769,24000.0,18000.0
...,...,...,...,...,...,...
295,h_95,518400,43200,275815.384615,259200.0,146880.0
296,h_96,66600,7400,33015.384615,29600.0,17760.0
297,h_97,250800,20900,83600.000000,62700.0,20900.0
298,h_98,226800,18900,96723.529412,75600.0,56700.0


## ばらつきの算出
分散/標準偏差  
分散、標準偏差の計算では（データ数-1）の値で割り算する部分があるが、データ数が1のときは0で割ることになり、Nullなどの不正な値が採用されてしまう。  
そのため、データ数が1のときは別の値が入るように設定することが望ましい。


In [12]:
# total_priceに対して、var関数とstd関数を適用し、分散値と標準偏差値を算出
result = reserve_tb \
  .groupby('hotel_id') \
  .agg({'total_price': ['var', 'std']}).reset_index()
result.columns = ['hotel_id', 'price_var', 'price_std']

# データ数が1件だったときは、分散値と標準偏差値がnaになっているので、0に置き換え
result.fillna({'price_var': 0, 'price_std': 0}, inplace=True)

In [13]:
result

,hotel_id,price_var,price_std
0,h_1,3.186549e+09,56449.526127
1,h_10,8.258133e+08,28736.968061
2,h_100,3.198316e+08,17883.835689
3,h_101,2.402441e+09,49014.703676
4,h_102,3.576923e+08,18912.755159
...,...,...,...
295,h_95,3.313772e+10,182037.696857
296,h_96,3.159231e+08,17774.225072
297,h_97,5.474685e+09,73991.116584
298,h_98,3.432893e+09,58590.896578


## 再頻出の算出

In [14]:
# round関数で四捨五入した後に、mode関数で最頻値を算出
reserve_tb['total_price'].round(-3).mode()

0    10000
1    20000
2    40000
Name: total_price, dtype: int64

## 順位の算出
`rank`関数を利用する。`rank`関数は文字列には対応していないので、timestamp型に変更する必要がある。

In [15]:
# rank関数で並び替えるために、データ型を文字列からtimestamp型に変換
# （「第10章 日時型」で解説）
reserve_tb['reserve_datetime'] = pd.to_datetime(
  reserve_tb['reserve_datetime'], format='%Y-%m-%d %H:%M:%S'
)

# log_noを新たな列として追加
# 集約単位の指定はgroup_byを利用
# 顧客ごとにまとめたreserve_datetimeを生成し、rank関数によって順位を生成
# ascendingをTrueにすることで昇順に設定(Falseだと降順に設定)
reserve_tb['log_no'] = reserve_tb \
  .groupby('customer_id')['reserve_datetime'] \
  .rank(ascending=True, method='first')

In [16]:
reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,log_no
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,1.0
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,2.0
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,3.0
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,4.0
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,5.0


### ランキング
ホテルごとの予約数に順位付けをする。同じ予約数の場合は、同予約数の全ホテルに最小のじゅにを付ける。

In [17]:
# 予約回数を計算(「3-1 データ数、種類数の算出」の例題を参照)
rsv_cnt_tb = reserve_tb.groupby('hotel_id').size().reset_index()
rsv_cnt_tb.columns = ['hotel_id', 'rsv_cnt']

# 予約回数をもとに順位を計算
# ascendingをFalseにすることで降順に指定
# methodをminに指定し、同じ値の場合は取り得る最小順位に指定
rsv_cnt_tb['rsv_cnt_rank'] = rsv_cnt_tb['rsv_cnt'] \
  .rank(ascending=False, method='min')

# 必要のないrsv_cntの列を削除
rsv_cnt_tb.drop('rsv_cnt', axis=1, inplace=True)

In [18]:
rsv_cnt_tb.head()

,hotel_id,rsv_cnt_rank
0,h_1,235.0
1,h_10,300.0
2,h_100,12.0
3,h_101,43.0
4,h_102,139.0
